In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-05-10 22:23:35.425495


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-05-10T22:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-05-10 22:00:00+0000', tz='UTC')

In [8]:
fetch_data_from

Timestamp('2025-04-11 22:00:00+0000', tz='UTC')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_bike_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year) and strip timezones (make tz-naive)
    historical_from_date = (from_date - timedelta(weeks=52)).replace(tzinfo=None)
    historical_to_date = (to_date - timedelta(weeks=52)).replace(tzinfo=None)

    # Load and process historical data
    rides_from = load_and_process_bike_data(year=historical_from_date.year, months=[historical_from_date.month])
    
    # Ensure pickup_datetime is tz-naive
    rides_from['pickup_datetime'] = pd.to_datetime(rides_from['pickup_datetime']).dt.tz_localize(None)
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_bike_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to['pickup_datetime'] = pd.to_datetime(rides_to['pickup_datetime']).dt.tz_localize(None)
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    # Simulate new production timestamps
    rides['simulated_datetime'] = rides['pickup_datetime'] + timedelta(weeks=52)

    # Sort by location and datetime
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides


In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-04.
Loading data for 2024-04...
Total records: 79,116
Valid records: 79,071
Records dropped: 45 (0.06%)
Successfully processed data for 2024-04.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-05.
Loading data for 2024-05...


/Users/ajit/Desktop/Citibike_prediction_aml/src/data_utils.py:98: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  location_filter = ~rides["start_station_id"].astype(str).str.contains(r'^\d+(\.\d+)?$', regex=True)
/Users/ajit/Desktop/Citibike_prediction_aml/src/data_utils.py:98: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  location_filter = ~rides["start_station_id"].astype(str).str.contains(r'^\d+(\.\d+)?$', regex=True)


Total records: 97,479
Valid records: 97,405
Records dropped: 74 (0.08%)
Successfully processed data for 2024-05.
Combining all monthly data...
Data loading and processing complete!


In [11]:
rides

,pickup_datetime,pickup_location_id,simulated_datetime
2,2024-04-12 22:01:23,HB101,2025-04-11 22:01:23
11,2024-04-12 22:07:06,HB101,2025-04-11 22:07:06
82,2024-04-12 23:07:29,HB101,2025-04-11 23:07:29
85,2024-04-12 23:08:09,HB101,2025-04-11 23:08:09
86,2024-04-12 23:08:36,HB101,2025-04-11 23:08:36
...,...,...,...
41410,2024-04-27 10:52:51,NaN,2025-04-26 10:52:51
42506,2024-04-27 15:45:23,NaN,2025-04-26 15:45:23
43062,2024-04-27 19:16:49,NaN,2025-04-26 19:16:49
45085,2024-04-28 15:01:41,NaN,2025-04-27 15:01:41


In [12]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [13]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2024-04-12 22:00:00,HB101,2
1,2024-04-12 23:00:00,HB101,6
2,2024-04-13 00:00:00,HB101,1
3,2024-04-13 01:00:00,HB101,1
4,2024-04-13 02:00:00,HB101,0
...,...,...,...
57763,2024-05-11 17:00:00,JC116,13
57764,2024-05-11 18:00:00,JC116,10
57765,2024-05-11 19:00:00,JC116,6
57766,2024-05-11 20:00:00,JC116,14


In [14]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57768 entries, 0 to 57767
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         57768 non-null  datetime64[ns]
 1   pickup_location_id  57768 non-null  object        
 2   rides               57768 non-null  int16         
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 1015.6+ KB


In [15]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-05-10 18:23:36,269 INFO: Initializing external client
2025-05-10 18:23:36,269 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 18:23:37,094 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000


In [16]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 57768/57768 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_citi_bike_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1231000/jobs/named/time_series_hourly_feature_group_citi_bike_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_citi_bike_1_offline_fg_materialization', 'SPARK'),
 None)